In [1]:
%load_ext dotenv
%dotenv

In [2]:
import numpy as np
import pandas as pd
import fastai
from fastcore.all import *
import os
from fastbook import *
from fastai.vision.widgets import *

In [3]:
creds = os.environ.get('KAGGLE_CREDS')

In [4]:
from pathlib import Path

cred_path = Path('~/.kaggle/kaggle.json').expanduser()
if not cred_path.exists():
    cred_path.parent.mkdir(exist_ok=True)
    cred_path.write_text(creds)
    cred_path.chmod(0o600)

In [5]:
path = Path('us-patent-phrase-to-phrase-matching')

In [6]:
if not path.exists():
    import zipfile,kaggle
    kaggle.api.competition_download_cli(str(path))
    zipfile.ZipFile(f'{path}.zip').extractall(path)

In [7]:
!dir {path}

 Volume in drive D is New Volume
 Volume Serial Number is DAFD-5401

 Directory of D:\PracticalDeepLearning\NLP Patent Phrase to Phrase Matching\us-patent-phrase-to-phrase-matching

20-06-2023  13:06    <DIR>          .
20-06-2023  18:25    <DIR>          ..
20-06-2023  13:06               693 sample_submission.csv
20-06-2023  13:06             1,965 test.csv
20-06-2023  13:06         2,141,136 train.csv
               3 File(s)      2,143,794 bytes
               2 Dir(s)  103,513,473,024 bytes free


In [8]:
df = pd.read_csv(path/'train.csv')
df

,id,anchor,target,context,score
0,37d61fd2272659b1,abatement,abatement of pollution,A47,0.50
1,7b9652b17b68b7a4,abatement,act of abating,A47,0.75
2,36d72442aefd8232,abatement,active catalyst,A47,0.25
3,5296b0c19e1ce60e,abatement,eliminating process,A47,0.50
4,54c1e3b9184cb5b6,abatement,forest region,A47,0.00
...,...,...,...,...,...
36468,8e1386cbefd7f245,wood article,wooden article,B44,1.00
36469,42d9e032d1cd3242,wood article,wooden box,B44,0.50
36470,208654ccb9e14fa3,wood article,wooden handle,B44,0.50
36471,756ec035e694722b,wood article,wooden material,B44,0.75


In [9]:
df.describe(include='object')

,id,anchor,target,context
count,36473,36473,36473,36473
unique,36473,733,29340,106
top,37d61fd2272659b1,component composite coating,composition,H01
freq,1,152,24,2186


In [10]:
df['input'] = 'TEXT1: ' + df.context + '; TEXT2: '+ df.target + '; ANC1: '+ df.anchor

In [11]:
df.input.head()

0    TEXT1: A47; TEXT2: abatement of pollution; ANC1: abatement
1            TEXT1: A47; TEXT2: act of abating; ANC1: abatement
2           TEXT1: A47; TEXT2: active catalyst; ANC1: abatement
3       TEXT1: A47; TEXT2: eliminating process; ANC1: abatement
4             TEXT1: A47; TEXT2: forest region; ANC1: abatement
Name: input, dtype: object

In [12]:
from datasets import Dataset, DatasetDict

ds = Dataset.from_pandas(df)

In [13]:
ds

Dataset({
    features: ['id', 'anchor', 'target', 'context', 'score', 'input'],
    num_rows: 36473
})

In [14]:
model_nm = 'microsoft/deberta-v3-small'

In [15]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

tokz = AutoTokenizer.from_pretrained(model_nm,use_fast=False)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [16]:
tokz.tokenize("G'day folks, I'm Bhaskar")

['▁G', "'", 'day', '▁folks', ',', '▁I', "'", 'm', '▁Bhaskar']

In [17]:
tokz.tokenize("A platypus is an ornithorhynchus anatinus.")

['▁A',
 '▁platypus',
 '▁is',
 '▁an',
 '▁or',
 'ni',
 'tho',
 'rhynch',
 'us',
 '▁an',
 'at',
 'inus',
 '.']

In [18]:
def tok_func(x): return tokz(x['input'])

In [19]:
tok_ds = ds.map(tok_func,batched=True)

Map:   0%|          | 0/36473 [00:00<?, ? examples/s]

In [20]:
row = tok_ds[0]
row['input'], row['input_ids']

('TEXT1: A47; TEXT2: abatement of pollution; ANC1: abatement',
 [1,
  54453,
  435,
  294,
  336,
  5753,
  346,
  54453,
  445,
  294,
  47284,
  265,
  6435,
  346,
  23702,
  435,
  294,
  47284,
  2])

In [21]:
tokz.vocab['▁of']

265

In [22]:
tok_ds = tok_ds.rename_columns({'score':'labels'})

In [23]:
eval_df = pd.read_csv(path/'test.csv')
eval_df.describe()

,id,anchor,target,context
count,36,36,36,36
unique,36,34,36,29
top,4112d61851461f60,el display,inorganic photoconductor drum,G02
freq,1,2,1,3


In [24]:
dds = tok_ds.train_test_split(0.25, seed=42)
dds

DatasetDict({
    train: Dataset({
        features: ['id', 'anchor', 'target', 'context', 'labels', 'input', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 27354
    })
    test: Dataset({
        features: ['id', 'anchor', 'target', 'context', 'labels', 'input', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 9119
    })
})

In [25]:
eval_df['input'] = 'TEXT1: ' + eval_df.context + '; TEXT2: ' + eval_df.target + '; ANC1: ' + eval_df.anchor

eval_ds = Dataset.from_pandas(eval_df).map(tok_func, batched=True)

Map:   0%|          | 0/36 [00:00<?, ? examples/s]

In [26]:
def corr(x,y): return np.corrcoef(x,y)[0][1]

In [27]:
def corr_d(eval_pred): return {'pearson':corr(*eval_pred)}

In [28]:
from transformers import TrainingArguments, Trainer

In [29]:
bs = 16
epochs = 5

In [30]:
lr = 8e-5

In [31]:
args = TrainingArguments('outputs', learning_rate=lr, warmup_ratio=0.1, lr_scheduler_type='cosine', fp16=True,
    evaluation_strategy="epoch", per_device_train_batch_size=bs, per_device_eval_batch_size=bs*2,
    num_train_epochs=epochs, weight_decay=0.01, report_to='none')

In [32]:
model = AutoModelForSequenceClassification.from_pretrained(model_nm,num_labels=1)
trainer = Trainer(model, args,train_dataset=dds['train'], eval_dataset=dds['test'],tokenizer=tokz, compute_metrics=corr_d)

Some weights of the model checkpoint at microsoft/deberta-v3-small were not used when initializing DebertaV2ForSequenceClassification: ['mask_predictions.classifier.weight', 'lm_predictions.lm_head.bias', 'mask_predictions.LayerNorm.weight', 'lm_predictions.lm_head.dense.bias', 'mask_predictions.dense.weight', 'mask_predictions.dense.bias', 'mask_predictions.LayerNorm.bias', 'mask_predictions.classifier.bias', 'lm_predictions.lm_head.LayerNorm.bias', 'lm_predictions.lm_head.LayerNorm.weight', 'lm_predictions.lm_head.dense.weight']
- This IS expected if you are initializing DebertaV2ForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaV2ForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from 

In [33]:
trainer.train()

C:\Python311\Lib\site-packages\transformers\optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Pearson
1,0.034800,0.026239,0.778369
2,0.024100,0.029699,0.794040
3,0.014600,0.021858,0.826463
4,0.009600,0.021718,0.833702
5,0.006700,0.021618,0.836318


TrainOutput(global_step=8550, training_loss=0.020184344534288374, metrics={'train_runtime': 6703.3809, 'train_samples_per_second': 20.403, 'train_steps_per_second': 1.275, 'total_flos': 799172955011700.0, 'train_loss': 0.020184344534288374, 'epoch': 5.0})

In [34]:
torch.cuda.empty_cache()


In [35]:
!nvidia-smi

Tue Jun 20 20:17:33 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 532.03                 Driver Version: 532.03       CUDA Version: 12.1     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                      TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf            Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce GTX 1650       WDDM | 00000000:01:00.0 Off |                  N/A |
| N/A   75C    P3               19W /  N/A|   2636MiB /  4096MiB |     99%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [36]:
preds = trainer.predict(eval_ds).predictions.astype(float)
preds

array([[ 0.27270508],
       [ 0.65234375],
       [ 0.50732422],
       [ 0.23376465],
       [-0.01971436],
       [ 0.48120117],
       [ 0.52636719],
       [-0.02494812],
       [ 0.20458984],
       [ 0.96826172],
       [ 0.25073242],
       [ 0.24743652],
       [ 0.80224609],
       [ 0.96923828],
       [ 0.78564453],
       [ 0.40917969],
       [ 0.26635742],
       [-0.0117569 ],
       [ 0.53857422],
       [ 0.35668945],
       [ 0.51855469],
       [ 0.24267578],
       [ 0.22924805],
       [ 0.2298584 ],
       [ 0.54785156],
       [-0.012146  ],
       [-0.01722717],
       [-0.00950623],
       [-0.00935364],
       [ 0.63134766],
       [ 0.30175781],
       [-0.01890564],
       [ 0.72265625],
       [ 0.5234375 ],
       [ 0.34228516],
       [ 0.22729492]])

In [37]:
preds = np.clip(preds,0,1)

In [38]:
preds


array([[0.27270508],
       [0.65234375],
       [0.50732422],
       [0.23376465],
       [0.        ],
       [0.48120117],
       [0.52636719],
       [0.        ],
       [0.20458984],
       [0.96826172],
       [0.25073242],
       [0.24743652],
       [0.80224609],
       [0.96923828],
       [0.78564453],
       [0.40917969],
       [0.26635742],
       [0.        ],
       [0.53857422],
       [0.35668945],
       [0.51855469],
       [0.24267578],
       [0.22924805],
       [0.2298584 ],
       [0.54785156],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.63134766],
       [0.30175781],
       [0.        ],
       [0.72265625],
       [0.5234375 ],
       [0.34228516],
       [0.22729492]])

In [40]:
import datasets

submission = datasets.Dataset.from_dict({
    'id': eval_ds['id'],
    'score': preds
})

submission.to_csv('submission.csv', index=False)

Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

1060